### 네이버 쇼핑 키워드 인사이트

#### 개인과제
- 키워드에 대한 검색량 데이터 수집
- 검색어에 대한 상품등록수 수집
- 수집한 데이터에 대한 인사이트 도출 : 예시) 경쟁률

#### 조별과제
- 스마트 스토어를 운영한다고 생각하고 경쟁력이 낮으면서 많은 수익을 낼수 있는 키워드와 상품 찾기
    - 예시외의 추가적인 페이지 데이터 수집하여 프로젝트 수행가능

#### 참고 서비스
- 신사임당 유투브 : https://youtu.be/Z0hL9khbpIw
- 응용 서비스 : https://whereispost.com/keyword, https://itemscout.io

#### 데이터 수집 프로세스
1. 네이버 검색광고 회원가입 및 로그인 : https://searchad.naver.com
2. 광고 시스템 > 도구 > API 사용관리
3. CUSTOMER_ID, API_KEY, SECRET_KEY 획득
4. 네이버 검색 광고 API 매뉴얼 사이트 참고 : http://naver.github.io/searchad-apidoc/#/guides
5. github 페이지 참고
    - https://github.com/naver/searchad-apidoc
    - https://github.com/naver/searchad-apidoc/blob/master/python-sample/examples/ad_management_sample.py
6. 키워드 입력하면 해당 키워드에 대한 데이터 수집

In [1]:
BASE_URL = 'https://api.naver.com'
API_KEY = '01000000000fcd1ce283f84b5b3f32330368ac6dce35c5a8e3feabf4cfec31ae32b7e9d0e8'
SECRET_KEY = 'AQAAAAAPzRzig/hLWz8yMwNorG3OPks1JGYBp40Ib+rqqydacA=='
CUSTOMER_ID = '2488417'

In [2]:
import time
import random
import requests

import signaturehelper


def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = signaturehelper.Signature.generate(timestamp, method, uri, SECRET_KEY)
    return {'Content-Type': 'application/json; charset=UTF-8', 'X-Timestamp': timestamp, 'X-API-KEY': API_KEY, 'X-Customer': str(CUSTOMER_ID), 'X-Signature': signature}

In [3]:
import hashlib
import hmac
import base64


class Signature:

    @staticmethod
    def generate(timestamp, method, uri, secret_key):
        message = "{}.{}.{}".format(timestamp, method, uri)
        hash = hmac.new(bytes(secret_key, "utf-8"), bytes(message, "utf-8"), hashlib.sha256)

        hash.hexdigest()
        return base64.b64encode(hash.digest())


- hint
    - github 페이지의 코드에서 signaturehelper.py 파일을 이용하여 header를 만들어야 합니다.

In [4]:
import requests, json, time
import pandas as pd
import signaturehelper
import warnings
warnings.filterwarnings('ignore')

#### 1. 개별과제 : 키워드에 대한 검색량 데이터 수집

In [5]:
#키워드 리스트에서 키워드 상세정보들을 datas리스트에 저장합니다.
def keyword_master(keywords):
    datas =[]
    for keyword in keywords:
        uri = '/keywordstool'
        method = 'GET'
        r = requests.get(BASE_URL + uri+f'?hintKeywords={keyword}&showDetail=1',
                         headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))
        datas.append(r.json()['keywordList'][0])
    return datas

In [6]:
#검색 키워드 작성
df = keyword_master(["헬스보충제",'모짜렐라치즈','자전거'])
#키워드들에 대한 세부정보 호출
keyword_df = pd.DataFrame(df)

## 결과

In [7]:
keyword_df.head()

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,monthlyAvePcClkCnt,monthlyAveMobileClkCnt,monthlyAvePcCtr,monthlyAveMobileCtr,plAvgDepth,compIdx
0,헬스보충제,1270,6970,19.7,191.7,1.80,3.17,15,높음
1,모짜렐라치즈,3150,30100,5.1,37.3,0.19,0.14,15,높음
2,자전거,36100,145400,171.7,1851.8,0.53,1.40,15,높음


#### 2. 개별과제 : 검색어에 대한 상품등록수 수집
- item_count : 등록된 상품수
- total_search_count : PC, Mobile 을 합친 검색량

In [8]:
from bs4 import BeautifulSoup as bs

In [9]:
def item_count(keywords):
    #데이터 프레임화하기 위한 리스트변수
    datas = []
    counts = []
    #예외 처리
    if type(keywords) is not list:
        print("리스트 형식으로 입력하세요")
        return
    
    #전체 상품 수 크롤링
    for keyword in keywords:
        url = f'https://search.shopping.naver.com/search/all?query={keyword}'
        response = requests.get(url)
        dom = bs(response.text,"html.parser")
        element = dom.select('.seller_filter_area > ul > li > a > span')[0].text.replace(',','')
        counts.append(element)
    
    #키워드 리스트 디테일 정보 추출
    details = keyword_master(keywords)
    
    #디테일 정보 부분화
    for i in range(len(details)):
        datas.append({
            'keyword' : details[i]['relKeyword'],
            'monthlyPcQcCnt' : details[i]['monthlyPcQcCnt'],
            'monthlyMobileQcCnt' : details[i]['monthlyMobileQcCnt'],
            'total_serch_count' : int(details[i]['monthlyPcQcCnt'])+int(details[i]['monthlyMobileQcCnt']),
            'item_count' : counts[i]
        })
    return datas

In [10]:
result_df = pd.DataFrame(item_count(['실내자전거','스피닝자전거','덤벨']))

## 결과 

In [11]:
result_df

,keyword,monthlyPcQcCnt,monthlyMobileQcCnt,total_serch_count,item_count
0,실내자전거,19400,163400,182800,255191
1,스피닝자전거,2330,24200,26530,63133
2,덤벨,7680,45500,53180,495822


#### 3. 개별과제 : 경쟁률
- 아이템수/검색수 : 높을수록 경쟁이 심한 상품 키워드 : **기본**
- 수집된 데이터에서 추가로 얻을수 있는 인사이트 추가 : **추가(1개 이상)**
    - 논리적인 인사이트이면 인사이트 수준에 대한 평가는 하지 않음

In [12]:
def item_count_compite(keywords):
    #데이터 프레임화하기 위한 리스트변수
    datas = []
    counts = []
    #예외 처리
    if type(keywords) is not list:
        print("리스트 형식으로 입력하세요")
        return
    
    #전체 상품 수 크롤링
    for keyword in keywords:
        url = f'https://search.shopping.naver.com/search/all?query={keyword}'
        response = requests.get(url)
        dom = bs(response.text,"html.parser")
        element = dom.select('.seller_filter_area > ul > li > a > span')[0].text.replace(',','')
        counts.append(element)
    
    #키워드 리스트 디테일 정보 추출
    details = keyword_master(keywords)
    
    #디테일 정보 부분화
    for i in range(len(details)):
        datas.append({
            'keyword' : details[i]['relKeyword'],
            'monthlyPcQcCnt' : details[i]['monthlyPcQcCnt'],
            'monthlyMobileQcCnt' : details[i]['monthlyMobileQcCnt'],
            'total_serch_count' : int(details[i]['monthlyPcQcCnt'])+int(details[i]['monthlyMobileQcCnt']),
            'item_count' : counts[i],
            'competition_rate' :   (int(counts[i])) / (int(details[i]['monthlyPcQcCnt'])+int(details[i]['monthlyMobileQcCnt'])),
        })
    return datas

In [13]:
result = item_count_compite(['실내자전거','스피닝자전거','덤벨','딥다라인','실내자전거렌탈'])

In [14]:
result_df = pd.DataFrame(result)
result_df

,keyword,monthlyPcQcCnt,monthlyMobileQcCnt,total_serch_count,item_count,competition_rate
0,실내자전거,19400,163400,182800,255191,1.396012
1,스피닝자전거,2330,24200,26530,63129,2.379533
2,덤벨,7680,45500,53180,495822,9.323467
3,딥다라인,330,3850,4180,55,0.013158
4,실내자전거렌탈,90,750,840,597,0.710714


In [15]:
# 딥다라인, 실내자전거렌탈의 키워드가 경쟁령이 낮다.

#### 4. 조별과제 : 스마트 스토어를 운영한다고 생각하고 경쟁력이 낮으면서 많은 수익을 낼수 있는 키워드 찾기
- 정답이 있는것은 아닙니다
- 여러가지 가정과 데이터를 수집, 분석하여 판매할 상품과 키워드를 선정해 보시면 됩니다.
- 앞서 2일간 했던 프로젝트와 같은 방법으로 수행하시면 됩니다.